In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import ast
import json
from collections import defaultdict
from matplotlib import pyplot as plt

import cv2
import numpy as np
import pandas as pd
import neuroglancer
from tqdm import tqdm
from skimage import io

HOME = os.path.expanduser("~")
DIR = os.path.join(HOME, 'programming/pipeline_utility/src')
sys.path.append(DIR)
#from utilities.file_location import FileLocationManager
#from src.utilities.sqlcontroller import SqlController
#from src.utilities.utilities_process import get_image_size

In [2]:
animal = 'DK55'
downsample_factor = 1
all_structures = False
section = 224

# OUTPUT_DIR_PATH = os.path.join(os.path.expanduser('~'))
OUTPUT_DIR_PATH = os.path.join('./')
CSV_DIR_PATH = f'/net/birdstore/Active_Atlas_Data/data_root/atlas_data/{animal}'
IMAGE_DIR_PATH = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps/CH3/full'
resolution = 0.325
aligned_shape = np.array((64000, 34000))
num_section = len(os.listdir(IMAGE_DIR_PATH))

downsampled_aligned_shape = np.round(aligned_shape / downsample_factor).astype(int)
scales = np.array([resolution * downsample_factor, resolution * downsample_factor, 20]) * 1000

In [ ]:
section

In [8]:
cshl_csvfile = 'cshl.premotor.csv'
dk_csvfile = 'dklabs.premotor.csv'
cshl_csvpath = os.path.join(CSV_DIR_PATH, cshl_csvfile)
dk_csvpath = os.path.join(CSV_DIR_PATH, dk_csvfile)
cshl_df = pd.read_csv(cshl_csvpath)
dk_df = pd.read_csv(dk_csvpath, header=None, names=['section', 'x', 'y'])

In [9]:
cshl_df.head()

,221,22359,21440
0,221,22359,21456
1,221,21751,21184
2,221,22039,21184
3,221,21815,21024
4,221,21815,21056


In [10]:
dk_df.head()

section             x        y
Layer    Description             X             Y  Section
Premotor NaN          36385.957031  19514.843750      124
         NaN          36418.734375  19700.191406      124
         NaN          41504.574219  14066.053711      128
         NaN          38170.566406  17708.964844      136

## Get the annotation points

In [ ]:
structures = ['premotor']
section_structure_vertices = defaultdict(dict)
vertices = []
for index, row in df.iterrows():
    SN = section
    #ST = row['structure']
    ST = 'premotor'
    x = row['x']
    y = row['y']
    vertices.append([x,y])
    #section_structure_vertices[SN][ST] = row['vertices']
section_structure_vertices[section]['premotor'] = vertices

In [ ]:
#section_structure_vertices

In [ ]:
print(section, downsampled_shape, downsample_factor)

## Reproduce create_clean transform

In [ ]:
section_offset = {}
#for file_name in tqdm(sorted(os.listdir(IMAGE_DIR_PATH))):
file_name = str(section).zfill(3) + '.tif'
filepath = os.path.join(IMAGE_DIR_PATH, file_name)
img = io.imread(filepath)
width, height = img.shape
downsampled_shape = np.round(np.array((width, height)) / downsample_factor)
section_offset[section] = (downsampled_aligned_shape - downsampled_shape) // 2

In [ ]:
print(section)
section_offset[section]

## Reproduce create_alignment transform

In [ ]:
def reverse_transform_create_alignment(points, transform):
    c = np.hstack((points, np.ones((points.shape[0], 1))))
    b = transform.copy()[:, 0:2] # Reverse rotation matrix by doing R^-1 = R^T
    b[2:, 0:2] = -transform[0:2, 2] # Reverse translation matrix by doing -T
    a = np.matmul(c, b)
    return a

def parse_elastix_parameter_file(filepath):
    """
    Parse elastix parameter result file.
    """
    def parameter_elastix_parameter_file_to_dict(filename):
        d = {}
        with open(filename, 'r') as f:
            for line in f.readlines():
                if line.startswith('('):
                    tokens = line[1:-2].split(' ')
                    key = tokens[0]
                    if len(tokens) > 2:
                        value = []
                        for v in tokens[1:]:
                            try:
                                value.append(float(v))
                            except ValueError:
                                value.append(v)
                    else:
                        v = tokens[1]
                        try:
                            value = (float(v))
                        except ValueError:
                            value = v
                    d[key] = value
            return d

    d = parameter_elastix_parameter_file_to_dict(filepath)

    # For alignment composition script
    rot_rad, x_mm, y_mm = d['TransformParameters']
    center = np.array(d['CenterOfRotationPoint']) / np.array(d['Spacing'])
    # center[1] = d['Size'][1] - center[1]

    xshift = x_mm / d['Spacing'][0]
    yshift = y_mm / d['Spacing'][1]

    R = np.array([[np.cos(rot_rad), -np.sin(rot_rad)],
                  [np.sin(rot_rad), np.cos(rot_rad)]])
    shift = center + (xshift, yshift) - np.dot(R, center)
    T = np.vstack([np.column_stack([R, shift]), [0, 0, 1]])
    return T


def load_consecutive_section_transform(animal, moving_fn, fixed_fn):
    """
    Load pairwise transform.

    Returns:
        (3,3)-array.
    """
    
    elastix_output_dir = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps/elastix'

    param_fp = os.path.join(elastix_output_dir, moving_fn + '_to_' + fixed_fn, 'TransformParameters.0.txt')
    #sys.stderr.write('Load elastix-computed transform: %s\n' % param_fp)
    if not os.path.exists(param_fp):
        raise Exception('Transform file does not exist: %s to %s, %s' % (moving_fn, fixed_fn, param_fp))
    transformation_to_previous_sec = parse_elastix_parameter_file(param_fp)

    return transformation_to_previous_sec

def parse_elastix(animal):
    """
    After the elastix job is done, this goes into each subdirectory and parses the Transformation.0.txt file
    Args:
        animal: the animal
    Returns: a dictionary of key=filename, value = coordinates
    """
    #fileLocationManager = FileLocationManager(animal)
    #DIR = fileLocationManager.prep
    DIR = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps'
    INPUT = os.path.join(DIR, 'CH1', 'thumbnail_cleaned')

    image_name_list = sorted(os.listdir(INPUT))
    anchor_idx = len(image_name_list) // 2
    # anchor_idx = len(image_name_list) - 1
    transformation_to_previous_sec = {}

    for i in range(1, len(image_name_list)):
        fixed_fn = os.path.splitext(image_name_list[i - 1])[0]
        moving_fn = os.path.splitext(image_name_list[i])[0]
        transformation_to_previous_sec[i] = load_consecutive_section_transform(animal, moving_fn, fixed_fn)

    transformation_to_anchor_sec = {}
    # Converts every transformation
    for moving_idx in range(len(image_name_list)):
        if moving_idx == anchor_idx:
            transformation_to_anchor_sec[image_name_list[moving_idx]] = np.eye(3)
        elif moving_idx < anchor_idx:
            T_composed = np.eye(3)
            for i in range(anchor_idx, moving_idx, -1):
                T_composed = np.dot(np.linalg.inv(transformation_to_previous_sec[i]), T_composed)
            transformation_to_anchor_sec[image_name_list[moving_idx]] = T_composed
        else:
            T_composed = np.eye(3)
            for i in range(anchor_idx + 1, moving_idx + 1):
                T_composed = np.dot(transformation_to_previous_sec[i], T_composed)
            transformation_to_anchor_sec[image_name_list[moving_idx]] = T_composed

    return transformation_to_anchor_sec

def create_warp_transforms(transforms, downsample_factor=32):
    def convert_2d_transform_forms(arr):
        return np.vstack([arr, [0, 0, 1]])
    
    transforms_scale_factor = 32 / downsample_factor 
    tf_mat_mult_factor = np.array([[1, 1, transforms_scale_factor], [1, 1, transforms_scale_factor]])
    transforms_to_anchor = {}
    for img_name, tf in transforms.items():
        transforms_to_anchor[img_name] = convert_2d_transform_forms(np.reshape(tf, (3, 3))[:2] * tf_mat_mult_factor) 

    return transforms_to_anchor


In [ ]:
transforms = parse_elastix(animal)
warp_transforms = create_warp_transforms(transforms, downsample_factor)
ordered_transforms = sorted(warp_transforms.items())

section_transform = {}
for filename, transform in ordered_transforms:
    section_num = int(filename.split('.')[0])
    transform = np.linalg.inv(transform)
    section_transform[section_num] = transform

## Alignment of annotation coordinates

In [ ]:
'''
(x', y') = (x * sx + y * ry + tx, x * rx + y * sy + ty)
'sx': T[0, 0], 'sy': T[1, 1], 'rx': T[1, 0], 'ry': T[0, 1], 'tx': T[0, 2], 'ty': T[1, 2]
'''
def transform_create_alignment(points, transform):
    a = np.hstack((points, np.ones((points.shape[0], 1))))
    b = transform.T[:, 0:2]
    c = np.matmul(a, b)
    return c

aligned_section_structure_polygons = defaultdict(dict)
for section in section_structure_vertices:
    for structure in section_structure_vertices[section]:
        points = np.array(section_structure_vertices[section][structure]) // downsample_factor
        points = points + section_offset[section] # create_clean offset
        points = transform_create_alignment(points, section_transform[section]) # create_alignment transform
        aligned_section_structure_polygons[section][structure] = [points]

In [ ]:
for k,v in aligned_section_structure_polygons.items():
    for k1,v1 in v.items():
        print(k,k1,len(v1[0]))
        points = v1[0]
df = pd.DataFrame(points, columns=['x','y'])
#df = df.loc[(df['y'] > 19000) & (df['y'] < 20000)]

df = df.round(0)
df = df.astype({'x': 'int32', 'y':'int32'})
df = df.sort_values(by=['x', 'y'])
df['section'] = section
outfile = f'/net/birdstore/Active_Atlas_Data/data_root/atlas_data/DK55/cshl2dk.{section}.csv'
df.to_csv(outfile, index=False)
df[['x','y','section']].head(10)

In [ ]:
infile = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data/DK55/premotor.dklabs.csv'
dk224df = pd.read_csv(infile, usecols=['Layer', 'X', 'Y', 'Section'])
dk224df = dk224df.loc[(dk224df['Layer'] == 'Premotor') & (dk224df['Section'] == section)]
#dk224df = dk224df.loc[(dk224df['Y'] > 19000) & (dk224df['Y'] < 20000)]
dk224df = dk224df.round(0)
dk224df = dk224df.astype({'X': 'int32', 'Y':'int32'})
dk224df = dk224df.sort_values(by=['Y', 'X'])
dk224df[['X','Y','Section']].head(10)

In [ ]:
import pickle

vertices_path = os.path.join(OUTPUT_DIR_PATH, f'{animal}_aligned_section_structure_polygons_down{downsample_factor}.pickle')
with open(vertices_path, 'wb') as file:
    pickle.dump(aligned_section_structure_polygons, file)

To this point, aligned_section_structure_polygons variable contains the aligned polygon vertices for each structure in each section. 
From now on, we introduce how to draw these points to numpy array or neuroglancer

In [ ]:
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
thickness = 2
colors = {}
colors['TR_CER'] = (0,255,0)
colors['FR_CER'] = (255,0,0)
colors['FL_CER'] = (0,255,0)
colors['MI_OVAL'] = (255,0,0)
#'VCA', 'VCP', 'DC'
unpad = lambda x: x[:, :-1]

PATH = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps/CH1'
DOWN16 = os.path.join(PATH, 'downsample_16')
ALIGNED = os.path.join(PATH, 'downsample_16_aligned')
DOWN32 = os.path.join(PATH, 'normalized')
ALIGNED32 = os.path.join(PATH, 'thumbnail_aligned')

In [ ]:
section = 221
file_name = f'{str(section).zfill(3)}.tif'
filepath = os.path.join(DOWN32, file_name)
img = cv2.imread(filepath)
section = int(file_name.split('.')[0])
radius = 2
color = (255,0,0)
for index, row in df.iterrows():
    x = row['x'] // 32
    y = row['y'] // 32
    print(x,y)
    cv2.circle(img, (x,y), radius, color, 2)

fig=plt.figure(figsize=(26,18), dpi= 100, facecolor='w', edgecolor='k')
plt.imshow(img, cmap="gray")
plt.title('Pre alignment section:{}'.format(section), fontsize=30)
plt.tick_params(axis='x', labelsize=30)
plt.tick_params(axis='y', labelsize=30)
plt.show()

In [ ]:
section

In [ ]:
#162, 185, 210
file_name = f'{str(section).zfill(3)}.tif'
filepath = os.path.join(DOWN32, file_name)
img = cv2.imread(filepath)
section = int(file_name.split('.')[0])
color = (0,255,0)
radius = 10
for structure in section_structure_vertices[section]:
    pts = section_structure_vertices[section][structure]
    points = np.array(pts, dtype=np.int32)
    offset = section_offset[section]
    transform = section_transform[section]
    
    points = reverse_transform_create_alignment(points, section_transform[section]) # reverse create_alignment transform
    points = (points - section_offset[section]).astype(np.int32) # reverse create_clean offset
    
    
    cx, cy = np.mean(points, axis=0)
    cx = int(cx // 32)
    cy = int(cy // 32)
    #print(structure,section,'with centers',cx,cy, 'offset', offset, pts)
    #cv2.polylines(img, [points], isClosed=True, color=colors[structure], thickness=2)
    cv2.circle(img, (cx,cy), radius, color, thickness)
    #cv2.putText(img, structure, (int(cx),int(cy)), font,1, colors[structure], 1, cv2.LINE_AA)

fig=plt.figure(figsize=(26,18), dpi= 100, facecolor='w', edgecolor='k')
plt.imshow(img, cmap="gray")
plt.title('Pre alignment section:{}'.format(section), fontsize=30)
plt.tick_params(axis='x', labelsize=30)
plt.tick_params(axis='y', labelsize=30)
plt.show()

# Fill up a pandas dataframe with the corrected vertices and save it.

In [ ]:
data = []

files = sorted(os.listdir(DOWN32))
for file_name in files:
    section = int(file_name.split('.')[0])
    
    if section in section_structure_vertices:
        for structure in section_structure_vertices[section]:
            pts = section_structure_vertices[section][structure]
            points = np.array(pts, dtype=np.int32)
            points = reverse_transform_create_alignment(points, section_transform[section]) # reverse create_alignment transform
            points = points - section_offset[section] # reverse create_clean offset
            data.append([structure, section, points])
            
df = pd.DataFrame(data=data, columns=['structure', 'section', 'vertices'])
outpath = os.path.join(CSV_DIR_PATH, f'{animal}_sections.162.185.210.csv')
df.to_csv(outpath, index=False)
redone_vertices = defaultdict(dict)
for index,row in df.iterrows():
    section = row['section']
    structure = row['structure']
    points = row['vertices']
    redone_vertices[section][structure] = points

In [ ]:
sections = [162, 185, 210]

font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
file_name = '210.tif'
filepath = os.path.join(DOWN32, file_name)
img = cv2.imread(filepath)
section = int(file_name.split('.')[0])

sl = []
for structure in section_structure_vertices[section]:
    pts = redone_vertices[section][structure]
    points = np.array(pts, dtype=np.int32)
    cx, cy = np.mean(points, axis=0)
    sl.append(structure)
    #print(structure,section,'with centers',cx,cy, 'offset', offset)
    cv2.polylines(img, [points], isClosed=True, color=colors[structure], thickness=2)
    cv2.putText(img, structure, (int(cx-5),int(cy-5)), font,
                1, colors[structure], 1, cv2.LINE_AA)

fig=plt.figure(figsize=(26,18), dpi= 100, facecolor='w', edgecolor='k')
plt.imshow(img, cmap="gray")
plt.title('MD589 at 1/32 size section:{}, structures {}'.format(section, sl), fontsize=20)
plt.tick_params(axis='x', labelsize=30)
plt.tick_params(axis='y', labelsize=30)
plt.show()
fig.savefig(f'/home/eddyod/Desktop/MD589.section{section}.jpg', bbox_inches='tight')